<a href="https://colab.research.google.com/github/polarboar/NER-on-MultiNERD/blob/main/RISE_Interview_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install transformers
!pip install transformers[torch]
!pip install evaluate
!pip install seqeval
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=71e80639dcb435808309edab097c12f87a1fab02b4c7e8873eddaab8bdaafc8d
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorForTokenClassification
import evaluate
import numpy as np
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

In [ ]:
dataset = load_dataset('Babelscape/multinerd')

print(dataset['train'][0])
label_to_id = {
    "O": 0,
    "B-PER": 1,
    "I-PER": 2,
    "B-ORG": 3,
    "I-ORG": 4,
    "B-LOC": 5,
    "I-LOC": 6,
    "B-ANIM": 7,
    "I-ANIM": 8,
    "B-BIO": 9,
    "I-BIO": 10,
    "B-CEL": 11,
    "I-CEL": 12,
    "B-DIS": 13,
    "I-DIS": 14,
    "B-EVE": 15,
    "I-EVE": 16,
    "B-FOOD": 17,
    "I-FOOD": 18,
    "B-INST": 19,
    "I-INST": 20,
    "B-MEDIA": 21,
    "I-MEDIA": 22,
    "B-MYTH": 23,
    "I-MYTH": 24,
    "B-PLANT": 25,
    "I-PLANT": 26,
    "B-TIME": 27,
    "I-TIME": 28,
    "B-VEHI": 29,
    "I-VEHI": 30,
  }

id_to_label = { label_to_id[key]: key for key in label_to_id }

# Only get english data
dataset = dataset.filter(lambda x: x['lang'] == 'en')

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

...

Generating test split: 0 examples [00:00, ? examples/s]

{'tokens': ['2002', 'ging', 'er', 'ins', 'Ausland', 'und', 'wechselte', 'für', '750.000', 'Pfund', 'Sterling', 'zu', 'Manchester', 'City', '.'], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 0], 'lang': 'de'}


Filter:   0%|          | 0/2678400 [00:00<?, ? examples/s]

Filter:   0%|          | 0/334800 [00:00<?, ? examples/s]

Filter:   0%|          | 0/335986 [00:00<?, ? examples/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
def tokenize_input_and_add_labels(input):
  labels = []
  tokens = tokenizer(input['tokens'], truncation=True, is_split_into_words=True)
  word_ids = tokens.word_ids()
  seen = {}
  for word_id in word_ids:
    if word_id == None:
      labels.append(-100)
    elif word_id in seen:
      labels.append(-100)
    else:
      labels.append(input['ner_tags'][word_id])
      seen[word_id] = True
  tokens['labels'] = labels
  return tokens

tokenized_dataset = dataset.map(tokenize_input_and_add_labels)

print(dataset['train'][0])
print(tokenized_dataset['train'][0])

from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

def compute_metrics(p):
  predictions, labels = p
  predictions = np.argmax(predictions, axis=2)

  true_predictions = [
      [id_to_label[p] for (p, l) in zip(prediction, label) if l != -100]
      for prediction, label in zip(predictions, labels)
  ]
  true_labels = [
      [id_to_label[l] for (p, l) in zip(prediction, label) if l != -100]
      for prediction, label in zip(predictions, labels)
  ]

  results = seqeval.compute(predictions=true_predictions, references=true_labels)
  return {
      "precision": results["overall_precision"],
      "recall": results["overall_recall"],
      "f1": results["overall_f1"],
      "accuracy": results["overall_accuracy"],
  }

seqeval = evaluate.load('seqeval')

model = AutoModelForTokenClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=len(id_to_label), id2label=id_to_label, label2id=label_to_id
)

training_args = TrainingArguments(
    output_dir = 'test_model',
    num_train_epochs = 2,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    load_best_model_at_end = True,
    push_to_hub = False
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_dataset['train'],
    eval_dataset = tokenized_dataset['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
    )

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/262560 [00:00<?, ? examples/s]

Map:   0%|          | 0/32820 [00:00<?, ? examples/s]

Map:   0%|          | 0/32908 [00:00<?, ? examples/s]

{'tokens': ['The', 'type', 'locality', 'is', 'Kīlauea', '.'], 'ner_tags': [0, 0, 0, 0, 5, 0], 'lang': 'en'}
{'tokens': ['The', 'type', 'locality', 'is', 'Kīlauea', '.'], 'ner_tags': [0, 0, 0, 0, 5, 0], 'lang': 'en', 'input_ids': [101, 1996, 2828, 10246, 2003, 11382, 17298, 5243, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 0, 0, 0, 0, 5, -100, -100, 0, -100]}


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.028200,0.053997,0.885613,0.894013,0.889793,0.982679
2,0.009300,0.069977,0.896311,0.905879,0.901070,0.983557


TrainOutput(global_step=65640, training_loss=0.029056586838218948, metrics={'train_runtime': 3960.3808, 'train_samples_per_second': 132.593, 'train_steps_per_second': 16.574, 'total_flos': 6435702579916416.0, 'train_loss': 0.029056586838218948, 'epoch': 2.0})

In [ ]:
trainer.evaluate(eval_dataset=tokenized_dataset['test'])

{'eval_loss': 0.04185158759355545,
 'eval_precision': 0.9130518160549227,
 'eval_recall': 0.92829011503217,
 'eval_f1': 0.9206079121389071,
 'eval_accuracy': 0.9869865283481017,
 'eval_runtime': 87.4362,
 'eval_samples_per_second': 376.366,
 'eval_steps_per_second': 47.051,
 'epoch': 2.0}